In [1]:
import uproot as ur 
import awkward as ak
import numpy as np
from sklearn.utils import shuffle
import pandas as pd

In [2]:
file = ur.open("fwdtree.root")
tree = file["fwd"]
branch_names = ["fstHits.mXYZ.fX", "fstHits.mXYZ.fY", "fstHits.mXYZ.fZ","reco.mIdTruth","seeds.mTrackId"]

# Load the branches into arrays
branches = tree.arrays(branch_names)

def read_tracks_from_branch(branch):
    track_list = []
    branch_ids = branch["reco.mIdTruth"]
    branch_trackid = branch["seeds.mTrackId"]
    for branch_id  in branch_ids:
        index_track = ak.where(branch_trackid == branch_id)[0]  # Ensure the indices are flattened
        pos_x, pos_y, pos_z = [], [], []
        for index in index_track:
            pos_x.append(branch["fstHits.mXYZ.fX"][index])
            pos_y.append(branch["fstHits.mXYZ.fY"][index])
            pos_z.append(branch["fstHits.mXYZ.fZ"][index])
        combined = list(zip(pos_x, pos_y, pos_z))
        track_list.append(combined)
    return track_list

def read_tracks_from_branches(branches):
    all_tracks = []
    for branch in branches:
        branch_tracks = read_tracks_from_branch(branch)
        all_tracks.extend(branch_tracks)
    return all_tracks

def append_true_track_feature(track_list):
    all_track = []
    for track in track_list:
        all_track.append(np.append(track,1))
    return np.array(all_track)

def sort_subarrays_by_z(arr):
    sorted_arr = []
    for subarray in arr:
        nan_mask = np.isnan(subarray).all(axis=1)
        non_nan_entries = subarray[~nan_mask]
        nan_entries = subarray[nan_mask]
        sorted_non_nan_entries = non_nan_entries[non_nan_entries[:, 2].argsort()]
        sorted_subarray = np.vstack((sorted_non_nan_entries, nan_entries))
        sorted_arr.append(sorted_subarray)
    return np.array(sorted_arr)

track_list = read_tracks_from_branches(branches)
for hits in track_list:
    padding_num = 3 - len(hits)
    for i in range(padding_num):
        hits.append((np.nan,np.nan,np.nan))

In [3]:
track_list = sort_subarrays_by_z(np.array(track_list))
# returns n fake track list, each track is defined as 3-3position in 3 layers of fst (9 points)
def make_fake_tracks_from_list(n,track_list,seed=0):
    fake_track_list = []
    random_min = 0
    random_max = len(track_list)
    np.random.seed(seed)
    while len(fake_track_list) < n:
        rng_track_indecies = np.random.randint(random_min,random_max,3)
        fake_layer_1 = track_list[rng_track_indecies[0]][0]
        fake_layer_2 = track_list[rng_track_indecies[1]][1]
        fake_layer_3 = track_list[rng_track_indecies[2]][2]
        fake_hit = np.concatenate((fake_layer_1,fake_layer_2,fake_layer_3),axis=0)  
        fake_track_list.append(np.append(fake_hit,0))
    return np.array(fake_track_list)
fake_track_list = make_fake_tracks_from_list(len(track_list),track_list)
mc_track_list = append_true_track_feature(track_list)
mc_and_fake_unshuffled = np.concatenate((fake_track_list,mc_track_list))
mc_and_fake_shuffled = shuffle(mc_and_fake_unshuffled)

training_frame = pd.DataFrame({'Truth':mc_and_fake_shuffled[:,-1],
                               'L1X':mc_and_fake_shuffled[:,0],'L1Y':mc_and_fake_shuffled[:,1],'L1Z':mc_and_fake_shuffled[:,2]
                               ,'L2X':mc_and_fake_shuffled[:,3],'L2Y':mc_and_fake_shuffled[:,4],'L2Z':mc_and_fake_shuffled[:,5]
                               ,'L3X':mc_and_fake_shuffled[:,6],'L3Y':mc_and_fake_shuffled[:,7],'L3Z':mc_and_fake_shuffled[:,8]
                               })
training_frame.head()
training_features = training_frame.drop("Truth",axis=1)
training_labels = training_frame["Truth"]


In [4]:
def linear_fit(row, col1, col2, col3):
    if pd.isna(row[col3]):
        return 2 * row[col2] - row[col1]
    return row[col3]

def linear_fit_implace(training_features):
    training_features['L3X'] = training_features.apply(lambda row: linear_fit(row, 'L1X','L2X','L3X'),axis=1)
    training_features['L3Y'] = training_features.apply(lambda row: linear_fit(row, 'L1Y','L2Y','L3Y'),axis=1)
    training_features['L3Z'] = training_features.apply(lambda row: linear_fit(row, 'L1Z','L2Z','L3Z'),axis=1)
    return training_features

training_features = linear_fit_implace(training_features)

# Cartesian XYZ

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(training_features, training_labels)
clf = DecisionTreeClassifier(random_state=0,max_depth=50,min_samples_split=10,min_samples_leaf=10,max_features='log2')
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.6910420475319927


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(training_features, training_labels)
clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.8086532602071907


In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(training_features, training_labels)
clf = GradientBoostingClassifier(random_state=0)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.7726995734308348


In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(training_features, training_labels)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
clf = MLPClassifier(
    hidden_layer_sizes=(50, 50),
    activation='relu',
    solver='adam',
    alpha=0.0001,
    batch_size='auto',
    learning_rate='constant',
    learning_rate_init=0.001,
    max_iter=1000,
    random_state=1
)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.7921998781230957


/Users/xihehan/anaconda3/envs/ml_env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:709: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


# Cartesian X,Y

In [ ]:
track_list = read_tracks_from_branches(branches)
for hits in track_list:
    padding_num = 3 - len(hits)
    for i in range(padding_num):
        hits.append((np.nan,np.nan,np.nan))
track_list = sort_subarrays_by_z(np.array(track_list))


In [ ]:
fake_track_list = make_fake_tracks_from_list(len(track_list),track_list)
mc_track_list = append_true_track_feature(track_list)
mc_and_fake_unshuffled = np.concatenate((fake_track_list,mc_track_list))
mc_and_fake_shuffled = shuffle(mc_and_fake_unshuffled)

training_frame = pd.DataFrame({'Truth':mc_and_fake_shuffled[:,-1],
                               'L1X':mc_and_fake_shuffled[:,0],'L1Y':mc_and_fake_shuffled[:,1],'L1Z':mc_and_fake_shuffled[:,2]
                               ,'L2X':mc_and_fake_shuffled[:,3],'L2Y':mc_and_fake_shuffled[:,4],'L2Z':mc_and_fake_shuffled[:,5]
                               ,'L3X':mc_and_fake_shuffled[:,6],'L3Y':mc_and_fake_shuffled[:,7],'L3Z':mc_and_fake_shuffled[:,8]
                               })
training_features = training_frame.drop(['Truth'],axis=1)
training_features = linear_fit_implace(training_features)
training_features = training_features.drop(['L1Z','L2Z','L3Z'],axis=1)
training_labels = training_frame["Truth"]


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(training_features, training_labels)
clf = DecisionTreeClassifier(random_state=0,max_depth=50,min_samples_split=10,min_samples_leaf=10,max_features='log2')
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.6587446678854357


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(training_features, training_labels)
clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.7397928092626447


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(training_features, training_labels)
clf = GradientBoostingClassifier(random_state=0)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.7422303473491774


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(training_features, training_labels)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
clf = MLPClassifier(
    hidden_layer_sizes=(50, 50),
    activation='relu',
    solver='adam',
    alpha=0.0001,
    batch_size='auto',
    learning_rate='constant',
    learning_rate_init=0.001,
    max_iter=1000,
    random_state=1
)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.7702620353443023


# Polar

In [ ]:
track_list = read_tracks_from_branches(branches)
for hits in track_list:
    padding_num = 3 - len(hits)
    for i in range(padding_num):
        hits.append((np.nan,np.nan,np.nan))
track_list = sort_subarrays_by_z(np.array(track_list))

training_features.head()

fake_track_list = make_fake_tracks_from_list(len(track_list),track_list)
mc_track_list = append_true_track_feature(track_list)
mc_and_fake_unshuffled = np.concatenate((fake_track_list,mc_track_list))
mc_and_fake_shuffled = shuffle(mc_and_fake_unshuffled)

training_frame = pd.DataFrame({'Truth':mc_and_fake_shuffled[:,-1],
                               'L1X':mc_and_fake_shuffled[:,0],'L1Y':mc_and_fake_shuffled[:,1],'L1Z':mc_and_fake_shuffled[:,2]
                               ,'L2X':mc_and_fake_shuffled[:,3],'L2Y':mc_and_fake_shuffled[:,4],'L2Z':mc_and_fake_shuffled[:,5]
                               ,'L3X':mc_and_fake_shuffled[:,6],'L3Y':mc_and_fake_shuffled[:,7],'L3Z':mc_and_fake_shuffled[:,8]
                               })
training_features = training_frame.drop(['Truth'],axis=1)
training_features = linear_fit_implace(training_features)
training_features = training_features.drop(['L1Z','L2Z','L3Z'],axis=1)
training_labels = training_frame["Truth"]

def cartesian_to_polar(row):
    R1 = np.sqrt(row['L1X']**2 + row['L1Y']**2)
    theta1 = np.arctan2(row['L1Y'],row['L1X'])
    R2 = np.sqrt(row['L2X']**2 + row['L2Y']**2)
    theta2 = np.arctan2(row['L2Y'],row['L2X'])
    R3 = np.sqrt(row['L3X']**2 + row['L3Y']**2)
    theta3 = np.arctan2(row['L3Y'],row['L3X'])
    return pd.Series([R1, theta1, R2, theta2, R3, theta3], index=['R1', 'theta1', 'R2', 'theta2', 'R3', 'theta3'])

training_features = training_features.apply(cartesian_to_polar, axis=1)
training_features.head()

,R1,theta1,R2,theta2,R3,theta3
0,9.3125,1.388764,9.312500,-1.552389,26.562500,-2.918654
1,12.1875,1.171961,26.562500,0.771081,6.437500,2.890020
2,17.9375,2.828661,9.312500,-2.517774,6.437500,-1.413308
3,9.3125,0.419288,9.312500,1.474667,9.312500,0.419288
4,26.5625,0.407016,20.812501,0.603366,16.412465,0.924603


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(training_features, training_labels)
clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.748933577087142


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(training_features, training_labels)
clf = DecisionTreeClassifier(random_state=0,max_depth=50,min_samples_split=10,min_samples_leaf=10,max_features='log2')
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.6904326630103595


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(training_features, training_labels)
clf = GradientBoostingClassifier(random_state=0)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.7349177330895795


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(training_features, training_labels)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
clf = MLPClassifier(
    hidden_layer_sizes=(50, 50),
    activation='relu',
    solver='adam',
    alpha=0.0001,
    batch_size='auto',
    learning_rate='constant',
    learning_rate_init=0.001,
    max_iter=1000,
    random_state=1
)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.753199268738574


# Cylindrical

In [ ]:
track_list = read_tracks_from_branches(branches)
for hits in track_list:
    padding_num = 3 - len(hits)
    for i in range(padding_num):
        hits.append((np.nan,np.nan,np.nan))
track_list = sort_subarrays_by_z(np.array(track_list))

training_features.head()

fake_track_list = make_fake_tracks_from_list(len(track_list),track_list)
mc_track_list = append_true_track_feature(track_list)
mc_and_fake_unshuffled = np.concatenate((fake_track_list,mc_track_list))
mc_and_fake_shuffled = shuffle(mc_and_fake_unshuffled)

training_frame = pd.DataFrame({'Truth':mc_and_fake_shuffled[:,-1],
                               'L1X':mc_and_fake_shuffled[:,0],'L1Y':mc_and_fake_shuffled[:,1],'L1Z':mc_and_fake_shuffled[:,2]
                               ,'L2X':mc_and_fake_shuffled[:,3],'L2Y':mc_and_fake_shuffled[:,4],'L2Z':mc_and_fake_shuffled[:,5]
                               ,'L3X':mc_and_fake_shuffled[:,6],'L3Y':mc_and_fake_shuffled[:,7],'L3Z':mc_and_fake_shuffled[:,8]
                               })
training_features = training_frame.drop(['Truth'],axis=1)
training_features = linear_fit_implace(training_features)
training_labels = training_frame["Truth"]

def cartesian_to_polar(row):
    R1 = np.sqrt(row['L1X']**2 + row['L1Y']**2)
    theta1 = np.arctan2(row['L1Y'],row['L1X'])
    R2 = np.sqrt(row['L2X']**2 + row['L2Y']**2)
    theta2 = np.arctan2(row['L2Y'],row['L2X'])
    R3 = np.sqrt(row['L3X']**2 + row['L3Y']**2)
    theta3 = np.arctan2(row['L3Y'],row['L3X'])
    return pd.Series([R1, theta1, row['L1Z'], R2, theta2, row['L2Z'], R3, theta3, row['L3Z']], index=['R1', 'theta1','Z1', 'R2', 'theta2','Z2', 'R3', 'theta3','Z3'])

training_features = training_features.apply(cartesian_to_polar, axis=1)
training_features.head()

,R1,theta1,Z1,R2,theta2,Z2,R3,theta3,Z3
0,9.3125,1.388764,168.745850,9.312500,-1.552389,182.278931,26.562500,-2.918654,180.190063
1,12.1875,1.171961,151.764038,26.562500,0.771081,180.883911,6.437500,2.890020,182.278931
2,17.9375,2.828661,153.852905,9.312500,-2.517774,165.262085,6.437500,-1.413308,165.262085
3,9.3125,0.419288,151.764038,9.312500,1.474667,151.764038,9.312500,0.419288,178.795044
4,26.5625,0.407016,153.158936,20.812501,0.603366,166.656982,16.412465,0.924603,180.155029


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(training_features, training_labels)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, n_jobs=-4, verbose=2)
grid_search.fit(X_train, y_train)
print("Best Hyperparameters:", grid_search.best_params_)
best_rf_model = grid_search.best_estimator_
tuned_rf_pred = best_rf_model.predict(X_test)
print("Tuned Random Forest Accuracy:", accuracy_score(y_test, tuned_rf_pred))
print("Tuned Random Forest Classification Report:")
print(classification_report(y_test, tuned_rf_pred))

0.8153564899451554
Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(training_features, training_labels)
clf = DecisionTreeClassifier(random_state=0,max_depth=50,min_samples_split=10,min_samples_leaf=10,max_features='log2')
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.7330895795246801


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(training_features, training_labels)
clf = GradientBoostingClassifier(random_state=0)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.7739183424741012


In [ ]:
œ

Test Accuracy: 0.77
